In [1]:
library(utils)
library(reshape2)
library(dplyr)
library(data.table)
library(caret)
library(ggplot2)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following objects are masked from ‘package:reshape2’:

    dcast, melt

Loading required package: lattice
Loading required package: ggplot2


In [2]:
a=read.delim("../analysis/GENIE3.top10_target_ranks.ranked_regulator_target_interactions.with_communities.correlations_and_directionality.tsv.gz",sep="\t",stringsAsFactors=FALSE)
nrow(a)
#a=a[a$community!="0",]
a=a[a$community %in% c(0,as.character(as.roman(1:11))),]
l=c("0",as.character(as.roman(1:11)))
a=transform(a,regulator_community=factor(a$regulator_community,levels=l),target_community=factor(a$target_community,levels=l))
nrow(a)
head(a)
table(a$community)

[1] 356457

[1] 356390

regulatoryGene,targetGene,full_cor,full_reg,our_cor,our_reg,weight,regulator_rank,target_rank,mutual_rank,regulator_community,target_community,community
Pp3c17_930.mRNA,Pp3s340_10.mRNA,0.1304851,+,1.874270e-15,+,0.06778052,1,1,1,I,I,I
Pp3c1_27430.pre_miRNA,Pp3c15_12100.mRNA,0.5589259,+,6.662628e-01,+,0.06724901,1,1,1,II,II,II
Pp3c17_17550.mRNA,Pp3c23_130.mRNA,0.8464697,+,8.916522e-01,+,0.06646932,1,1,1,II,II,II
Pp3c17_18920.pre_miRNA,Pp3s32_420.mRNA,1.0000000,+,1.000000e+00,+,0.06632740,1,1,1,III,III,III
Pp3c15_549.mRNA,Pp3c4_24040.mRNA,1.0000000,+,1.000000e+00,+,0.06400505,1,1,1,IV,IV,IV
Pp3c4_3072.pre_miRNA,Pp3c22_1240.mRNA,0.9517601,+,9.665704e-01,+,0.06353888,1,1,1,II,II,II



     0      I     II    III     IV     IX      V     VI    VII   VIII      X 
141355  19019  22867  10543  13303  36095  22025  12309  26584  22113  17330 
    XI 
 12847 

In [3]:
n=read.delim("../analysis/GENIE3.top10_target_ranks.communities_PLM.with_node_centralities.csv",sep=";",stringsAsFactors=FALSE)
n=n[n$community %in% c(0,as.character(as.roman(1:11))),]
n=transform(n,community=factor(n$community,levels=l[-1]))
head(n)
table(n$community)

geneid,community,degree_centrality,betweenness_centrality,eigenvector_centrality,local_reaching_centrality,centrality_rank
Pp3c27_3080.mRNA,I,109,0,0.00000000,0.9977499,1
Pp3c11_23290.mRNA,I,552,3190,0.05359921,0.9971070,2
Pp3c7_2300.mRNA,I,505,7529,0.02518149,0.9971070,3
Pp3c17_23620.mRNA,I,490,1988,0.05419710,0.9971070,4
Pp3c16_13280.mRNA,I,485,9982,0.05634423,0.9971070,5
Pp3c15_11610.mRNA,I,461,3734,0.04049888,0.9971070,6



   I   II  III   IV    V   VI  VII VIII   IX    X   XI 
3112 4418 1705 2728 3435 1849 3851 3662 6167 2403 2309 

In [4]:
n$is_regulator=n$geneid %in% unique(a$regulatoryGene)

In [5]:
regr=read.delim("TAP_or_named.tsv",sep="\t",stringsAsFactors=FALSE)
head(regr)
n$is_TF=n$geneid %in% subset(regr,TAP_class=="TF")$geneid
table(n$is_regulator,n$is_TF)

geneid,spectype,origin,name,superfamily,inferred_by_rule,TAP_class,combined_name
Pp3c1_10451.pre_miRNA,pre_miRNA,mirbase,"MI0006045,ppt-MIR1079",,,,"MI0006045,ppt-MIR1079"
Pp3c1_10620.mRNA,mRNA,cosmoss,"mscl12,MSCL12",,,,"mscl12,MSCL12"
Pp3c1_110.mRNA,mRNA,cosmoss,,OFP,True,TR,OFP
Pp3c1_11190.mRNA,mRNA,cosmoss,,TRAF,True,TR,TRAF
Pp3c1_11420.mRNA,mRNA,cosmoss,NOG-1,,,,NOG-1
Pp3c1_1170.mRNA,mRNA,cosmoss,"hdma1504,HDMA1504",SWI/SNF_SWI3,True,TR,"hdma1504,HDMA1504"


       
        FALSE  TRUE
  FALSE 33970     0
  TRUE    695   974

In [6]:
d=read.delim(gzfile("DEK1_study.DEGs.minimal_info.tsv.gz"),sep="\t",row.names=1)
head(d)

,spectype,origin,name,superfamily,TAP_class,defline,deltaDEK_vs_OE.b_wt,WT_vs_deltaDEK.b_wt,WT_vs_LG3.b_wt,WT_vs_Loop.b_wt,⋯,elongation_caulonema.late.reduced_vs_normal.qval_lrt,gametophore_formation.aberrant_vs_normal.qval_lrt,gametophore_formation.arrested_vs_normal.qval_lrt,gametophore_formation.delayed_vs_normal.qval_lrt,number_buds_per_filament.high_vs_normal.qval_lrt,percents_filaments_with_buds.high_vs_normal.qval_lrt,percents_filaments_with_buds.low_vs_normal.qval_lrt,protonemata_branching.aberrant_vs_normal.qval_lrt,rhizoid_formation.delayed_vs_normal.qval_lrt,rhizoid_formation.ectopic_vs_normal.qval_lrt
Pp3c1_100.mRNA,mRNA,cosmoss,,NA,NA,PTHR19139//PTHR19139:SF147 - AQUAPORIN TRANSPORTER // SUBFAMILY NOT NAMED (1 of 1),-0.14447429,0.03519676,0.262197137,0.07392628,⋯,0.3369932,0.2364591,0.5566207,1.000000000,0.9484040,0.7356034,0.7026984,0.2035819,1.878378e-01,0.7446988
Pp3c1_1000.mRNA,mRNA,cosmoss,,NA,NA,,-0.27292998,-0.05173580,-0.009327494,-0.11139047,⋯,0.5332108,0.4095824,0.3752901,0.001433834,0.9769126,0.5500804,0.4910445,0.3866815,4.058102e-05,0.3464745
Pp3c1_10000.mRNA,mRNA,cosmoss,,NA,NA,K00021 - hydroxymethylglutaryl-CoA reductase (NADPH) (HMGCR) (1 of 3),0.09853629,-0.12750588,0.144333576,-0.05030005,⋯,1.0000000,0.4095824,1.0000000,0.653042712,0.2606938,0.6395874,0.8204697,0.8653970,1.000000e+00,0.5573633
Pp3c1_10010.mRNA,mRNA,cosmoss,,NA,NA,"PTHR11451:SF41 - GLUTAMATE--TRNA LIGASE, MITOCHONDRIAL-RELATED (1 of 1)",-0.20581712,0.03086996,0.019516436,-0.07224085,⋯,0.6329936,0.6527880,1.0000000,0.164413823,0.4195042,1.0000000,0.7382343,0.4440858,4.620523e-02,1.0000000
Pp3c1_10040.mRNA,mRNA,cosmoss,,NA,NA,K14846 - ribosome production factor 1 (RPF1) (1 of 1),-0.17333214,-0.01953257,0.088532877,-0.02856157,⋯,0.5752269,0.1459987,0.4990066,0.021573836,0.3754352,0.6495691,0.8001721,0.3658798,4.540952e-04,0.2623199
Pp3c1_10050.mRNA,mRNA,cosmoss,,NA,NA,3.1.1.56 - Methylumbelliferyl-acetate deacetylase / Esterase D (1 of 1),0.33416986,-0.18787943,0.317312864,-0.14652969,⋯,0.5644653,0.3874502,0.5136794,0.091799361,0.2434924,NA,0.5881345,0.4013159,1.340674e-01,0.5693843


In [7]:
#names(d)[29:33]
names(d)[7:11]

[1] "deltaDEK_vs_OE.b_wt" "WT_vs_deltaDEK.b_wt" "WT_vs_LG3.b_wt"     
[4] "WT_vs_Loop.b_wt"     "WT_vs_OE.b_wt"

In [8]:
e=read.delim("calpain_cleavage_prediction.cosmoss_V3.3.release.pep.ccd.NERD_classification.tsv",sep="\t",row.names=1)
names(e)
levels(e$fate)
e=data.table(geneid=row.names(e),e[,c(6,7,19,20,23:24)])
ee=setdiff(n$geneid,e$geneid)
ee=data.table(geneid=ee,secondary_ATE_raw=0,secondary_peptidase_raw=0,uncertainty=1,U1=0,fate=factor(rep("unchanged",length(ee)),levels=levels(e$fate)),fate_strict=factor(rep("unchanged",length(ee)),levels=levels(e$fate)))
e=rbind(e,ee)
setkey(e,"geneid")
head(e)

[1] "id"                                  "sites_raw"                          
 [3] "len"                                 "tertiary_deaminated_raw"            
 [5] "tertiary_oxidized_or_acetylated_raw" "secondary_ATE_raw"                  
 [7] "secondary_peptidase_raw"             "primary_acetylated_raw"             
 [9] "ratio"                               "lratio"                             
[11] "SLC"                                 "sites_rel"                          
[13] "tertiary_deaminated_rel"             "tertiary_oxidized_or_acetylated_rel"
[15] "secondary_ATE_rel"                   "secondary_peptidase_rel"            
[17] "primary_acetylated_rel"              "clust"                              
[19] "uncertainty"                         "U1"                                 
[21] "U2"                                  "U3"                                 
[23] "fate"                                "fate_strict"

[1] "NERD"      "other"     "unchanged"

geneid,secondary_ATE_raw,secondary_peptidase_raw,uncertainty,U1,fate,fate_strict
Pp3c10_100.ncRNA,0,0,1.000000e+00,0.000000e+00,unchanged,unchanged
Pp3c10_10040.mRNA,0,11,5.693259e-05,9.999431e-01,NERD,NERD
Pp3c10_10090.mRNA,1,6,6.172489e-05,6.172489e-05,NERD,NERD
Pp3c10_10095.C_D_box_snoRNA,0,0,1.000000e+00,0.000000e+00,unchanged,unchanged
Pp3c10_10100.mRNA,0,6,4.176609e-03,4.176609e-03,NERD,NERD
Pp3c10_10130.mRNA,0,3,2.393614e-02,2.393520e-02,other,other


In [9]:
p=read.delim("profile_phases.tsv",sep="\t",stringsAsFactors=FALSE)
head(p)
names(p)[2]="geneid"
p=p[,c(2,1,5)]
unique(p$phase)
pp=setdiff(n$geneid,p$geneid)
p=rbind(p,data.frame(geneid=pp,profile=0,phase="unchanged"))
p$phase=factor(p$phase,c("unchanged","profile1_early","profile1_late","profile1_uniform","profile2_early","profile2_late","profile2_uniform"))
table(p$profile,p$phase)
head(p)

profile,id,cluster,expr,phase
1,Pp3c1_10200.mRNA,1,late,profile1_late
1,Pp3c1_10220.mRNA,1,late,profile1_late
1,Pp3c1_11030.mRNA,2,early,profile1_early
1,Pp3c1_11190.mRNA,2,early,profile1_early
1,Pp3c1_11820.mRNA,3,uniform,profile1_uniform
1,Pp3c1_11980.mRNA,3,uniform,profile1_uniform


[1] "profile1_late"    "profile1_early"   "profile1_uniform" "profile2_uniform"
[5] "profile2_late"    "profile2_early"

   
    unchanged profile1_early profile1_late profile1_uniform profile2_early
  0     30555              0             0                0              0
  1         0            958           943              738              0
  2         0              0             0                0            667
   
    profile2_late profile2_uniform
  0             0                0
  1             0                0
  2          1019              759

geneid,profile,phase
Pp3c1_10200.mRNA,1,profile1_late
Pp3c1_10220.mRNA,1,profile1_late
Pp3c1_11030.mRNA,1,profile1_early
Pp3c1_11190.mRNA,1,profile1_early
Pp3c1_11820.mRNA,1,profile1_uniform
Pp3c1_11980.mRNA,1,profile1_uniform


In [10]:
dereg=data.frame(deregulation=rowSums(abs(d[,c(7,8,11)])), 
                 significant=(d$deltaDEK_vs_OE.qval_lrt<0.01| d$WT_vs_OE.qval_lrt<0.1) & d$WT_vs_deltaDEK.qval_lrt<0.1)
nr=setdiff(n$geneid,row.names(dereg))
nonreg=data.frame(deregulation=rep(0,length(nr)),significant=FALSE)
row.names(nonreg)=nr
rm(nr)
dereg=rbind(dereg,nonreg)
dereg$deregulation[is.na(dereg$deregulation)]=0
dereg$significant[is.na(dereg$significant)]=FALSE

nrow(dereg)
dereg=data.frame(geneid=row.names(dereg),dereg)
row.names(dereg)=NULL
dereg=merge(dereg,n[,c("geneid","is_regulator","is_TF")],by="geneid")
dereg=merge(dereg,e[,c("geneid","uncertainty","U1","secondary_ATE_raw","secondary_peptidase_raw","fate_strict")],by="geneid")
dereg=merge(dereg,p,by="geneid")
dereg=na.omit(dereg)
nrow(dereg)
#names(dereg)[1]="geneid"
dereg=as.data.table(dereg)
setkey(dereg,"geneid")
head(dereg)

[1] 35639

[1] 35639

geneid,deregulation,significant,is_regulator,is_TF,uncertainty,U1,secondary_ATE_raw,secondary_peptidase_raw,fate_strict,profile,phase
Pp3c1_100.mRNA,0.2762754,FALSE,FALSE,FALSE,0.00000000,0.0000000,0,0,NERD,0,unchanged
Pp3c1_1000.mRNA,0.6631450,FALSE,FALSE,FALSE,0.36506797,0.6349320,0,6,other,0,unchanged
Pp3c1_10000.mRNA,0.2532188,FALSE,FALSE,FALSE,0.02949464,0.0294939,0,6,other,0,unchanged
Pp3c1_10010.mRNA,0.4157642,FALSE,FALSE,FALSE,0.09267909,0.0926765,1,2,other,0,unchanged
Pp3c1_10020.seryl_tRNA,0.0000000,FALSE,FALSE,FALSE,1.00000000,0.0000000,0,0,unchanged,0,unchanged
Pp3c1_10040.mRNA,0.3886370,FALSE,FALSE,FALSE,0.06555740,0.9344426,2,3,other,0,unchanged


In [23]:
write.table(dereg,file="deregulation.tsv",sep="\t",row.names=FALSE,col.names=TRUE,quote=FALSE)

In [11]:
table(dereg$uncertainty<0.05 & dereg$U1>0.9, dereg$fate_strict)

       
         NERD other unchanged
  FALSE  2613 19195      4808
  TRUE   9023     0         0

In [12]:
A=as.data.table(a)
setkeyv(A,c("regulatoryGene","targetGene"))

In [13]:
nrow(A)

[1] 356390

In [14]:
A=merge(A,dereg,by.x="targetGene",by.y="geneid",all.X=TRUE,all.y=FALSE)
A=merge(A,dereg,by.x="regulatoryGene",by.y="geneid",all.X=TRUE,all.y=FALSE,suffixes=c("_tar","_reg"))
head(A)

regulatoryGene,targetGene,full_cor,full_reg,our_cor,our_reg,weight,regulator_rank,target_rank,mutual_rank,⋯,significant_reg,is_regulator_reg,is_TF_reg,uncertainty_reg,U1_reg,secondary_ATE_raw_reg,secondary_peptidase_raw_reg,fate_strict_reg,profile_reg,phase_reg
Pp3c1_10451.pre_miRNA,Pp3c1_32930.mRNA,0.7007387,+,5.835452e-15,+,0.03492560,6,1,2.449490,⋯,FALSE,TRUE,FALSE,1,0,0,0,unchanged,0,unchanged
Pp3c1_10451.pre_miRNA,Pp3c1_9450.mRNA,0.3117556,+,5.458784e-01,+,0.01663924,26,4,10.198039,⋯,FALSE,TRUE,FALSE,1,0,0,0,unchanged,0,unchanged
Pp3c1_10451.pre_miRNA,Pp3c10_12940.mRNA,0.3965461,+,5.835452e-15,+,0.01030016,51,5,15.968719,⋯,FALSE,TRUE,FALSE,1,0,0,0,unchanged,0,unchanged
Pp3c1_10451.pre_miRNA,Pp3c10_19830.mRNA,0.4312062,+,5.835452e-15,+,0.01163123,43,5,14.662878,⋯,FALSE,TRUE,FALSE,1,0,0,0,unchanged,0,unchanged
Pp3c1_10451.pre_miRNA,Pp3c11_22730.mRNA,0.4777171,+,5.835452e-15,+,0.01862452,23,4,9.591663,⋯,FALSE,TRUE,FALSE,1,0,0,0,unchanged,0,unchanged
Pp3c1_10451.pre_miRNA,Pp3c11_8100.mRNA,0.3117556,+,5.458784e-01,+,0.01812989,24,3,8.485281,⋯,FALSE,TRUE,FALSE,1,0,0,0,unchanged,0,unchanged


In [15]:
nrow(A)

[1] 356390

In [16]:
A$fate_strict_reg=relevel(A$fate_strict_reg,"unchanged")
A$community=ordered(A$community,levels=c("0",as.character(as.roman(1:11))))

In [17]:
#A$put_DEK1_target=A$fate_strict_reg=="NERD" &  (A$secondary_peptidase_raw_reg>0 | A$secondary_ATE_raw_reg>0 ) &  A$significant_tar==TRUE
A$put_DEK1_target=A$fate_strict_reg=="NERD" &  A$U1_reg>0.9 & A$uncertainty_reg<0.05  &  A$significant_tar==TRUE

In [18]:
table(A$community,A$put_DEK1_target)

      
        FALSE   TRUE
  0    125019  16336
  I     16325   2694
  II    20201   2666
  III   10352    191
  IV    13174    129
  V     18304   3721
  VI    11862    447
  VII   23793   2791
  VIII  19687   2426
  IX    28888   7207
  X     15580   1750
  XI    11949    898

In [19]:
AA=droplevels(subset(A,is_TF_reg==TRUE))

In [24]:
write.table(AA,file="TF_targets.NERD_classification.tsv",sep="\t",row.names=FALSE,col.names=TRUE,quote=FALSE)

In [22]:
round(prop.table(table(AA$community,AA$put_DEK1_target),1)*100)

      
       FALSE TRUE
  0       86   14
  I       85   15
  II      88   12
  III     98    2
  IV      97    3
  V       78   22
  VI      95    5
  VII     90   10
  VIII    85   15
  IX      74   26
  X       89   11
  XI      92    8

In [20]:
t(AA[1,])

regulatoryGene,Pp3c1_12940.mRNA
targetGene,Pp3c1_10010.mRNA
full_cor,-0.6258329
full_reg,-
our_cor,-0.7894632
our_reg,-
weight,0.01369468
regulator_rank,71
target_rank,5
mutual_rank,18.84144
regulator_community,IX
